## Bacterie for k=121

In [62]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import pulp
import time
from ortools.sat.python import cp_model
import matplotlib.colors as mcolors
from utility import *
from reconstruit import *
from tools_base import *
from tools import *

### Hamiltonian

### In this section let us describe the Hamiltonian that we used. We recall the constraint.


1-for all node i , $ \sum_{j\in i_{\text{out}}} x_{i,j}=1$, where $i_{\text{out}}$ denotes the outgoing nodes connedcted to the node i

2-for all node j, $ \sum_{i\in j_{\text{in}}} x_{i,j}=1$, where $j_{\text{in}}$ denotes the incoming nodes connedcted to the node j

3- for all node i, $ \sum_{k\in i_{\text{out}}} x_{i,k}=\sum_{ j\in i_{in} } x_{j,i}$ 

4- the Hamiltonian is $ H=-\sum_{(i,j)\in E} x_{i,j} $, where $E$ is the set of edges

All in all, we have
\begin{align}
H &=-\delta\sum_{(i,j)\in E} x_{i,j} +\alpha \sum_{i}\biggl( \sum_{ j\in i_{\text{out}} } x_{i,j}-1\biggr)^2+\beta \sum_{j}\biggl(  \sum_{i\in j_{\text{in}} } x_{i,j}-1 \biggr)^2\\
&+\gamma \sum_{i}\biggl(  \sum_{k\in i_{\text{out}} }  x_{i,k} -\sum_{j\in i_{\text{in}} } x_{j,i} \biggr)^2
\end{align}

After developing the precedent expression, we have
\begin{align}
H=&-\delta \sum_{(i,j)\in E} x_{i,j} +(\gamma-\alpha)\sum_{i}\sum_{j\in i_{\text{out}} } x_{i,j}+(\alpha+\gamma)\sum_{i}\sum_{\substack{k\neq j\\ k,j\in i_{\text{out}} }} x_{i,j}x_{i,k}\\
& (\gamma-\beta)\sum_{j}\sum_{i\in j_{\text{in}} }x_{i,j}+(\gamma+\beta)\sum_{j}\sum_{\substack{ i\neq k\\ i,k\in j_{\text{in}} } } x_{i,j}x_{k,j}-2\gamma\sum_{i}\sum_{ \substack{ k\in i_{\text{out}}\\ j\in i_{\text{in}} } } x_{i,k}x_{j,i}
\end{align}
     

In [ ]:
In this section let us describe the Hamiltonian that we used. We recall the constraint.
 
     \begin{enumerate}
     \item for all node i , $ \sum_{j\in i_{\text{out}}} x_{i,j}=1$, where $i_{\text{out}}$ denotes the outgoing nodes connedcted to the node i
     \item for all node j, $ \sum_{i\in j_{\text{in}}} x_{i,j}=1$, where $j_{\text{in}}$ denotes the incoming nodes connedcted to the node j
    \item for all node i, $ \sum_{k\in i_{\text{out}}} x_{i,k}=\sum_{ j\in i_{in} } x_{j,i}$ 
        \item the Hamiltonian is $ H=-\sum_{(i,j)\in E} x_{i,j} $, where $E$ is the set of edges
     \end{enumerate}

     All in all, we have
     \begin{align}
     H &=-\delta\sum_{(i,j)\in E} x_{i,j} +\alpha \sum_{i}\biggl( \sum_{ j\in i_{\text{out}} } x_{i,j}-1\biggr)^2+\beta \sum_{j}\biggl(  \sum_{i\in j_{\text{in}} } x_{i,j}-1 \biggr)^2\\
     &+\gamma \sum_{i}\biggl(  \sum_{k\in i_{\text{out}} }  x_{i,k} -\sum_{j\in i_{\text{in}} } x_{j,i} \biggr)^2
     \end{align}

     After developing the precedent expression, we have
     \begin{align}
     H=&-\delta \sum_{(i,j)\in E} x_{i,j} +(\gamma-\alpha)\sum_{i}\sum_{j\in i_{\text{out}} } x_{i,j}+(\alpha+\gamma)\sum_{i}\sum_{k\neq j\\ k,j\in i_{\text{out}} } x_{i,j}x_{i,k}\\
     & (\gamma-\beta)\sum_{j}\sum_{i\in j_{\text{in}} }x_{i,j}+(\gamma+\beta)\sum_{j}\sum_{i\neq k\\ i,k\in j_{\text{in}}} x_{i,j}x_{k,j}-2\gamma\sum_{i}\sum_{k\in i_{\text{out}}\\ j\in i_{\text{in}} } x_{i,k}x_{j,i}
     \end{align}

In [2]:
# Prepare and use the biggest component
verbose=False
filename="graph_bacteria"
path="spneumoniae_k_121.unitigs.fa"
G_bacteria = generate_de_bruijn_graph_from_fa(path,verbose)
save_graph_to_html(G_bacteria,filename+"_de-Bruijn")
remove_isolated_nodes(G_bacteria,verbose)
DG_bacteria = convert_de_bruijn_to_digraph(G_bacteria,verbose)

SG_bacteria = find_biggest_component(DG_bacteria)

In [3]:
# AFFICHAGE Des GRAPHEs DECOMPOSEE
liste= decomposition(SG_bacteria)
l=len(liste)
colors=generate_colors(l)


net = Network(notebook=False, directed=True)
for i,graphe in enumerate(liste):
    for node in graphe.nodes():
        net.add_node(node, color="lightblue")
    for edge in graphe.edges():
        net.add_edge(edge[0], edge[1], color=colors[i])

net.write_html('sub_graphs_bacteria.html')

In [23]:
# Some preparation
nodes_in=[node for node,d in SG_bacteria.in_degree() if d==0] 
node_0=nodes_in[0]
list_sb_bac=decomposition(SG_bacteria)
list_bride_node_bac=bridge_graph(SG_bacteria)

registre_shared_node_fr_subgraph=sharing_nodes(list_bride_node_bac,list_sb_bac)
registre_shared_subgraph_fr_node=sharing_subgraph(list_bride_node_bac,list_sb_bac)
regi_shared_nodes,regi_shared_subgraph=sharing_node_subgraph(list_bride_node_bac,list_sb_bac)

l=len(list_sb_bac)
len_regi_shared_shared_node_fr_subgraph=len( registre_shared_node_fr_subgraph)
len_regi_shared_subgraph_from_node=len( registre_shared_subgraph_fr_node)
len_shared_nodes=len( regi_shared_nodes) #same as len( regi_shared_subgraph)



#t=True


#liste_node_end=[]
#k=0    


In [ ]:
################## Essai global
# looking for graph containing the start node
# To change the start node in the liste nodes_in
GLP=nx.DiGraph()
#node_start= node_0
node_start='c2987'
liste_graphe_used=[]
n_iter=0

list_gamma=[30,35,40,45,25,20,27,48,42,38,15,12,100]
while node_start !="" and n_iter < l:
    print("n_iter=", n_iter, "start_node=", node_start)
    for graphe in list( set(list_sb_bac)-set(liste_graphe_used)) :
        if graphe.has_node(node_start):
            break

    # CHoice of the next node
    list_bridge_node_ly_graphe=registre_shared_subgraph_fr_node[graphe] #contain all bridge node of graphe
    list_bridge_node_out_ly_graphe=[] #contain all out bridge node of graphe
    for node in list_bridge_node_ly_graphe :
        if graphe.out_degree(node)==0:
            list_bridge_node_out_ly_graphe.append(node)

    if list_bridge_node_out_ly_graphe !=[]:
        liste_subgraphe_ly_node=[]
        for node_1 in list_bridge_node_out_ly_graphe:
            ll= registre_shared_node_fr_subgraph[node_1] #liste graphe shared node
        
            for subgraphe_2 in ll:
                if subgraphe_2 not in liste_graphe_used and  subgraphe_2 !=graphe :
                    lll_node= registre_shared_subgraph_fr_node[subgraphe_2]
                    list_bridge_node_out_ly_graphe_2=[]
                
                    for node_2 in lll_node :
                        if subgraphe_2.out_degree(node_2)==0:
                            list_bridge_node_out_ly_graphe_2.append(node_1)
                

    # We used graphe                
    liste_graphe_used.append(graphe)        


    # end_node        
    if list_bridge_node_out_ly_graphe_2 !=[]:
        node_end= random.choice(list_bridge_node_out_ly_graphe_2) #We choice randomly  one value 
    elif list_bridge_node_out_ly_graphe !=[]:
        node_end=random.choice(list_bridge_node_out_ly_graphe)  #We choice randomly  one value 
    else:
        node_end=""


    #Resolution 
    for gama in list_gamma:
        # Resulotion with Or-Tool
        xx,_=opti_or_tool(graphe,start_node=node_start,end_node=node_end,alpha=10,beta=10,gamma=gama,delta=10, limit_time=None)
        
        #Construction of the output graph 
        G_sub_graph=nx.DiGraph()
        edges_graphe= list(graphe.edges() )
        len_graphe= len( edges_graphe)

        for i in range(len_graphe):
            if xx[i]==1:
                 G_sub_graph.add_node((edges_graphe[i])[0])
                 G_sub_graph.add_node((edges_graphe[i])[1])
                 G_sub_graph.add_edge((edges_graphe[i])[0],(edges_graphe[i])[1])

        #Check whether the output is a path
        booleen= check_path(  G_sub_graph )
        if booleen: # if True
            print(booleen)
            break # we out the boucle for gama in list_gamma

    #GLP.add_nodes_from( G_sub_graph.nodes() ) 
    #GLP.add_edges_from( G_sub_graph.edges() )
    
    if booleen==False:
        print("The last out isn't a path") 
        break
    # Updated GLP
    GLP.add_nodes_from( G_sub_graph.nodes() ) 
    GLP.add_edges_from( G_sub_graph.edges() )


    

    
        

    node_start=  node_end     
    n_iter+=1

In [ ]:
# AFFICHAGE DU GRAPHE AVEC LE LONGEST PATH
#net = Network(notebook=False, directed=True)
# Ajouter SG en bleu
#for node in SG_bacteria.nodes():
#    net.add_node(node, color="lightblue")
#for edge in SG_bacteria.edges():
#    net.add_edge(edge[0], edge[1], color="blue")

# Ajouter GLP en rouge
#for node in GLP.nodes():
#    net.add_node(node, color="pink")
#for edge in GLP.edges():
#    net.add_edge(edge[0], edge[1], color="red")   

#net.write_html('graph_genom_k_121_combi_paths.html')

# AFFICHAGE DU GRAPHE AVEC LE LONGEST PATH
#save_graph_to_html(GLP,'graph_genom_k_121_combi_paths.html')

net = Network(notebook=False, directed=True)
# Ajouter SG en bleu
for node in SG_bacteria.nodes():
    if node in GLP.nodes():
        net.add_node(node, color="#438F68")
    else :
        net.add_node(node, color="#84B8D9")
for edge in SG_bacteria.edges():
    if edge in GLP.edges():
        net.add_edge(edge[0], edge[1], color="#5A9C56")   
    else :
        net.add_edge(edge[0], edge[1], color="#84B8D9")
 # Optional: show physics options in the browser
net.show_buttons(filter_=['physics'])
net.write_html('graph_genom_k_121_combi_paths.html')




In [25]:
# Reconstruction of the sequence
k=120
list_GLP=recontruct_path(GLP) # ordered list of nodes of GLP
contigs=recontruct_DNA(SG_bacteria,list_GLP, k)
#len( GLP.nodes())

 ### Determination of set of contigs

We are going to find a set of contigs

In [49]:
# Some preparation
nodes_in=[node for node,d in SG_bacteria.in_degree() if d==0] 
node_0=nodes_in[0]
list_sb_bac=decomposition(SG_bacteria)
list_bride_node_bac=bridge_graph(SG_bacteria)

registre_shared_node_fr_subgraph=sharing_nodes(list_bride_node_bac,list_sb_bac)
registre_shared_subgraph_fr_node=sharing_subgraph(list_bride_node_bac,list_sb_bac)
regi_shared_nodes,regi_shared_subgraph=sharing_node_subgraph(list_bride_node_bac,list_sb_bac)

l_nodes_in=len(nodes_in)
l=len(list_sb_bac)
len_regi_shared_shared_node_fr_subgraph=len( registre_shared_node_fr_subgraph)
len_regi_shared_subgraph_from_node=len( registre_shared_subgraph_fr_node)
len_shared_nodes=len( regi_shared_nodes) #same as len( regi_shared_subgraph)



#t=True


#liste_node_end=[]
#k=0    


In [50]:
################## Essai global
start_time=time.time() # time of execution
k=120 # (k-1)mers
set_contigs=[]
set_paths=[]

for node_start in nodes_in:
    GLP=nx.DiGraph()
    liste_graphe_used=[]
    
    list_gamma=[30,35,40,45,25,20,27,48,42,38,15,12,100]
    list_alpha=[20,10]
    while node_start !="" and n_iter < l:
        #print("n_iter=", n_iter, "start_node=", node_start)
        for graphe in list( set(list_sb_bac)-set(liste_graphe_used)) :
            if graphe.has_node(node_start):
                break

        # CHoice of the next node
        list_bridge_node_ly_graphe=registre_shared_subgraph_fr_node[graphe] #contain all bridge node of graphe
        list_bridge_node_out_ly_graphe=[] #contain all out bridge node of graphe
        for node in list_bridge_node_ly_graphe :
            if graphe.out_degree(node)==0:
                list_bridge_node_out_ly_graphe.append(node)

        if list_bridge_node_out_ly_graphe !=[]:
            liste_subgraphe_ly_node=[]
            for node_1 in list_bridge_node_out_ly_graphe:
                ll= registre_shared_node_fr_subgraph[node_1] #liste graphe shared node
        
                for subgraphe_2 in ll:
                    if subgraphe_2 not in liste_graphe_used and  subgraphe_2 !=graphe :
                        lll_node= registre_shared_subgraph_fr_node[subgraphe_2]
                        list_bridge_node_out_ly_graphe_2=[]
                
                        for node_2 in lll_node :
                            if subgraphe_2.out_degree(node_2)==0:
                                list_bridge_node_out_ly_graphe_2.append(node_1)
                

        # We used graphe                
        liste_graphe_used.append(graphe)        


        # end_node        
        if list_bridge_node_out_ly_graphe_2 !=[]:
            node_end= random.choice(list_bridge_node_out_ly_graphe_2) #We choice randomly  one value 
        elif list_bridge_node_out_ly_graphe !=[]:
            node_end=random.choice(list_bridge_node_out_ly_graphe)  #We choice randomly  one value 
        else:
            node_end=""


        #Resolution 
        for gama in list_gamma:
            for alfa in list_alpha:
                # Resulotion with Or-Tool
                xx,_=opti_or_tool(graphe,start_node=node_start,end_node=node_end,alpha=alfa,beta=alfa,gamma=gama,delta=10, limit_time=None)
        
                #Construction of the output graph 
                G_sub_graph=nx.DiGraph()
                edges_graphe= list(graphe.edges() )
                len_graphe= len( edges_graphe)

                for i in range(len_graphe):
                    if xx[i]==1:
                         G_sub_graph.add_node((edges_graphe[i])[0])
                         G_sub_graph.add_node((edges_graphe[i])[1])
                         G_sub_graph.add_edge((edges_graphe[i])[0],(edges_graphe[i])[1])

                #Check whether the output is a path
                booleen= check_path(  G_sub_graph )
                if booleen: # if True
                #print(booleen)
                
                    # Updated GLP
                    GLP.add_nodes_from( G_sub_graph.nodes() ) 
                    GLP.add_edges_from( G_sub_graph.edges() )
                    set_paths.append(GLP)
                
                    # Construction of contigs
                    list_GLP=recontruct_path(GLP) # ordered list of nodes of GLP
                    contigs=recontruct_DNA(SG_bacteria,list_GLP, k)
                    set_contigs.append(contigs)
                    break # for alfa in list_alpha

            if booleen: 
                break # We out of the boucle for gama in list_gamma

        #arret ici
        if booleen==False: # The algorithm doesn't find any path
            #print("The last out isn't a path") 
            set_contigs.append("")
            set_paths.append("")
            print(node_start)
            break

        # Updated the started node
        
set_end_nodes=[]
        node_start=  node_end    

end_time=time.time()    

### Affichage des contigs

In [51]:
# Time of execution
print("time execution:", end_time-start_time,"secondes")

time execution: 4.1922619342803955 secondes


### Affichage contigs

In [61]:
while True:
    try:
        n = int(input(f"Entrez un entier entre {0} et {l_nodes_in-1}: "))
        print(f"One the longest path starting from : {nodes_in[n]}  has {len(set_contigs[n] )}. Its sequence is:" )
        print("\n", set_contigs[n])
        break   # on sort de la boucle si l'entrée est correcte
    except ValueError:
        print("Erreur : veuillez entrer un entier valide.")


Entrez un entier entre 0 et 46:  46


One the longest path starting from : c770  has 11023. Its sequence is:

 CTACTGGAACAAGTTATCCTGCAATCAATGATTATAATTTTAATCTATTATTAATTGCTCTCCCCCCACTATCCGAACAACAACGAATAGTAGAAGCAATCGAATCAGCTTTAGAAAAAGTAGATGAATATGCTGAAAGTTATAATAGACTAGAACAGCTAGATAAAGAATTTCCAGATAAACTAAAAAAATCTATTCTTCAATATGCTATGCAAGGAAAATTAGTTGAACAAGACCCAAATGATGAATCAGTCGAAGTTTTACTTGAAAAAATACGAGCAGAAAAACAAAAACTCTTTGAAGAAGGCAAGATTAAAAAGAAAGATTTGGACATTTCTATTGTTTCCCAAGGAGATGATAACTCTTATTATGGGAATATACCTATGAATTGGGTTGTTATAAAAATAAAAGATATTTTTTCAATGAATACAGGTCTTTCTTACAAGAAGGGCGATTTAAGCATTAATAATAAAGGTGTTAGAATTATACGTGGTGGTAATATTAAGCCTTTAGAATTTTCTCTGTTGGATAATGATTACTACATTGATACACAATTCATCTCCTCTGAACAAGTTTATTTAAAACATAATCAGCTAATAACACCTGTATCAACCTCTTTAGAACATATTGGAAAGTTTGCAAGAATCGATAAAGACTATGATGGTGTTGTGGCTGGTGGATTTATTTTCCAATTAACACCATTCGAAAGTTCAGAGATTATTTCAAAATTTCTATTATTTAACTTGTCTTCTCCGTTATTTTATAAACAATTGAAAGCAATAACTAAACTATCAGGTCAAGCTTTATATAATATTCCTAAAACTACACTGAGCGAGCTATTAATTCCGTTAGCTCCTTTTGAGGAACAGGAACTTATTACTCAAAAAGTTGAGAAACTTTTTGAAAAAGTAAATCAACTTTGGAAAT